# The Knapesack Problem with the genetic algorithm

In [11]:
# import the necessary packages
import random
import numpy as np
import matplotlib.pyplot as plt

## Define the required functions

### Create The Population

In [12]:
# create propulation function
def create_population(population_size, array_size):
    population = []
    for i in range(population_size):
        population.append(list(np.random.randint(2, size=array_size)))
    return np.array(population)


### Fitness And Selcetion Functions

In [13]:
# fitness function

def fitness(array, max_weight_bag ,items):
    w,v = 0 , 0
    for i in range(len(array)):
        if array[i] ==1:
            v+=items[i].get('value')
            w+=items[i].get('weight')
    if w <= max_weight_bag:
        return v
    else:
        return 0

# selection function 
# Roulette wheel selection
def roulittete_selection(population, fitness, max_weight_bag,items):
    fitness_array = np.array([fitness(population[i],max_weight_bag,items) for i in range(len(population))])
    prop_fitness = fitness_array / np.sum(fitness_array)
    cum_prop_fitness = np.cumsum(prop_fitness)
    random_number = random.randint(0, 98)/100
    for i, cum_prop in enumerate(cum_prop_fitness):
        if random_number <= cum_prop :
            return population[i]
        

### Crossover Functions

In [14]:
# the single crossover function 

def single_crossover(array1, array2):
    # create a random index
    index = random.randint(0, len(array1))
    # perform the crossover
    offspring1 = np.copy(array1)
    offspring2 = np.copy(array2)
    temp1 = np.copy(offspring1[index:])
    temp2 = np.copy(offspring2[index:])
    offspring1[index:] = temp2
    offspring2[index:] = temp1
    
    return offspring1, offspring2


# the double crossover function

def double_crossover(array1, array2):
    # create 2 random indexes not similar to each other
    index1 = random.randint(0, len(array1) - 1)
    index2 = random.randint(0, len(array1) - 1)
    while index1 == index2:
        index2 = random.randint(0, len(array1) - 1)
    # make sure index1 < index2
    if index1 > index2:
        index1, index2 = index2, index1
    # perform the crossover
    offspring1 = np.copy(array1)
    offspring2 = np.copy(array2)
    temp1 = np.copy(offspring1[index1:index2])
    temp2 = np.copy(offspring2[index1:index2])
    offspring1[index1:index2] = temp2
    offspring2[index1:index2] = temp1
    return offspring1, offspring2


# the unifrom crossover function

def uniform_crossover(array1, array2):  
    if len(array1)%2==0:
        part1 = np.ones(int(len(array1)/2))
        part2 = np.zeros(int(len(array1)/2))
        mask = np.concatenate((part1, part2))
        np.random.shuffle(mask)
    else:
        mask = np.random.randint(2, size=len(array1))
    offspring1 = np.zeros(len(array1))
    offspring2 = np.zeros(len(array1))
    for i in range(len(array1)):
        if mask[i] == 1:
            offspring1[i] = array1[i]
            offspring2[i] = array2[i]
        else:
            offspring1[i] = array2[i]
            offspring2[i] = array1[i]
    return offspring1, offspring2


### Mutation Functions

In [15]:
# the invert mutation function

def invert_bit(my_array, mutation_rate):
    array = np.copy(my_array)
    random_number = random.randint(0, 100)/100
    if random_number < mutation_rate:
        # select a random index
        index = random.randint(0, len(array) - 1)
        # invert the array
        array[index] = 1 - array[index]
    return array


# switch 2 bits mutation function

def switch_bit(my_array, mutation_rate):
    array = np.copy(my_array)
    random_number = random.randint(0, 100)/100
    if random_number < mutation_rate:
        # select 2 random indexes
        index1 = random.randint(0, len(array) - 1)
        index2 = random.randint(0, len(array) - 1)
        while index1 == index2:
            index2 = random.randint(0, len(array) - 1)
        if index1 > index2:
            index1, index2 = index2, index1
        # switch the values
        array[index1], array[index2] = array[index2], array[index1]
    return array


# the shuffle element in between

def shuffle_element(my_array, mutation_rate):           
    array = np.copy(my_array)
    random_number = random.randint(0, 100)/100
    if random_number < mutation_rate:
        # select 2 random indexes
        index1 = random.randint(0, len(array) - 1)
        index2 = random.randint(0, len(array) - 1)
        while index1 == index2:
            index2 = random.randint(0, len(array) - 1)
        if index1 > index2:
            index1, index2 = index2, index1
        # shuffle the values
        array[index1:index2] = np.random.permutation(array[index1:index2])
    return array

### Elite Selection


In [16]:
# keep the best chromosomes 

def next_generation(population, elit_size, fitness,max_weight_bag,items):
    # create the fitness array
    fitness_array = [fitness(population[i],max_weight_bag,items) for i in range(len(population))]
    # sort the fitness array descendingly and get the indexes
    sorted_index = np.argsort(fitness_array)[::-1]
    new_population = []
    for i in range(elit_size):
        new_population.append(list(population[sorted_index[i]]))
    return new_population


### Generitic Algorithm

In [17]:
# the main genetic algorithm function

def NapSack_genetic_algorithm(items, max_weight_bag, population_size, elit_size, generations, mutation_rate, next_generation, fitness, selection_func, cross_over_func, mutation_func ):
       
    # create the initial population
    population = create_population(population_size, len(items))
    # loop through the generations
    for i in range(generations):
        # create the new population
        new_population = []
        # create the new population using the elitism (exploitation)
        next_gen_arr=next_generation(population, elit_size, fitness,max_weight_bag,items)
        new_population.extend(next_gen_arr)
        # create the remaining population
        while len(new_population) < population_size:
            # select the parents
            parent1 = selection_func(population, fitness, max_weight_bag,items)
            parent2 = selection_func(population, fitness, max_weight_bag,items)
            while np.array_equal(parent1, parent2):
                parent2 = selection_func(population, fitness, max_weight_bag,items)
            # perform the crossover (exploration)
            offspring1, offspring2 = cross_over_func(parent1, parent2)
            # perform the mutation  (exploration)
            offspring1 = mutation_func(offspring1, mutation_rate)
            offspring2 = mutation_func(offspring2, mutation_rate)
            # add the offspring to the new population
            new_population.append(offspring1)
            new_population.append(offspring2)
        # convert the new population to a numpy array
        population = np.array(new_population)
        if (np.array_equal(next_generation(population, 3, fitness,max_weight_bag,items),next_gen_arr)) and (i>=10):
            break
    # return the best solution
    return next_generation(population, 1, fitness,max_weight_bag,items)[0] , i

## Examine the algorithim

In [18]:

# define the  items to choose from 
items = [
    {'weight': 2, 'value': 3},
    {'weight': 3, 'value': 4},
    {'weight': 4, 'value': 5},
    {'weight': 5, 'value': 8},
    {'weight': 9, 'value': 10}
]

# define the genetic algorithm parameters
max_weight_bag = 20
population_size = 10
elit_size = int(population_size * 0.3)
generations = 100
mutation_rate = 0.02

# define the function parameters
selection_func = roulittete_selection
cross_over_func = double_crossover
mutation_func = invert_bit

# run the genetic algorithm
KnapeSack=NapSack_genetic_algorithm(items, max_weight_bag, population_size, elit_size,generations, mutation_rate,
                                next_generation, fitness,selection_func, cross_over_func, mutation_func )


# print the results
print("the Napsack should contain",KnapeSack[0])
print("the convergence happened after ",KnapeSack[1], "iterations")


# the total value of the bag is 
v,w=0,0
for i in range(len(KnapeSack[0])):
    if KnapeSack[0][i]!=0:
        v+=items[i].get("value")
        w+=items[i].get("weight")

print("the total value of the bag is ", v)
print("the total weight of the bag is ", w)

the Napsack should contain [1, 1, 0, 1, 1]
the convergence happened after  10 iterations
the total value of the bag is  25
the total weight of the bag is  19


In [19]:
items

[{'weight': 2, 'value': 3},
 {'weight': 3, 'value': 4},
 {'weight': 4, 'value': 5},
 {'weight': 5, 'value': 8},
 {'weight': 9, 'value': 10}]

In [20]:
# try the greedy method to solve the napsack problem 
# define the value to the weight list 

ratio = [items[i].get("value")/items[i].get("weight") for i in range (len(items))]
ratio

[1.5, 1.3333333333333333, 1.25, 1.6, 1.1111111111111112]